In [2]:
!pip install tensorflow

     -------------------------------------- 266.3/266.3 MB 6.4 MB/s eta 0:00:00
     -------------------------------------- 439.2/439.2 kB 9.1 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 11.8 MB/s eta 0:00:00
     ---------------------------------------- 3.7/3.7 MB 11.4 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB ? eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 10.7 MB/s eta 0:00:00
     ---------------------------------------- 23.2/23.2 MB 7.4 MB/s eta 0:00:00
     -------------------------------------- 895.9/895.9 kB 7.1 MB/s eta 0:00:00
     ---------------------------------------- 65.5/65.5 kB ? eta 0:00:00
     -------------------------------------- 126.5/126.5 kB 7.3 MB/s eta 0:00:00
     ---------------------------------------- 6.0/6.0 MB 11.0 MB/s eta 0:00:00
     ------------------------------------- 177.8/177.8 kB 11.2 MB/s eta 0:00:00
     ------------------------------------- 781.3/781.3 kB 

In [12]:
import numpy as np
import pandas as pd
import json
import csv
import random
  
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
  
import pprint
import tensorflow.compat.v1 as tf
from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
tf.disable_eager_execution()
  
# Reading the data
data = pd.read_csv('C:/Users/zrowx/OneDrive/바탕 화면/hackathon/news.csv')
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [13]:

# encoding the labels
le = preprocessing.LabelEncoder()
le.fit(data['label'])
data['label'] = le.transform(data['label'])

In [14]:

embedding_dim = 50
max_length = 54
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_size = 3000
test_portion = .1

In [15]:

title = []
text = []
labels = []
for x in range(training_size):
    title.append(data['title'][x])
    text.append(data['text'][x])
    labels.append(data['label'][x])

In [19]:

tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(title)
word_index1 = tokenizer1.word_index
vocab_size1 = len(word_index1)
sequences1 = tokenizer1.texts_to_sequences(title)
padded1 = pad_sequences(
    sequences1,  padding=padding_type, truncating=trunc_type)
split = int(test_portion * training_size)
training_sequences1 = padded1[split:training_size]
test_sequences1 = padded1[0:split]
test_labels = labels[0:split]
training_labels = labels[split:training_size]
embeddings_index = {}
with open("C:/Users/zrowx/OneDrive/바탕 화면/hackathon/glove.6B.50d.txt", encoding='UTF-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
  
# Generating embeddings
embeddings_matrix = np.zeros((vocab_size1+1, embedding_dim))
for word, i in word_index1.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector

In [20]:

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size1+1, embedding_dim,
                              input_length=max_length, weights=[
                                  embeddings_matrix],
                              trainable=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',
              optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 54, 50)            377600    
                                                                 
 dropout (Dropout)           (None, 54, 50)            0         
                                                                 
 conv1d (Conv1D)             (None, 50, 64)            16064     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 12, 64)           0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                        

In [21]:

num_epochs = 50
  
training_padded = np.array(training_sequences1)
training_labels = np.array(training_labels)
testing_padded = np.array(test_sequences1)
testing_labels = np.array(test_labels)
  
history = model.fit(training_padded, training_labels, 
                    epochs=num_epochs,
                    validation_data=(testing_padded,
                                     testing_labels), 
                    verbose=2)

Train on 2700 samples, validate on 300 samples
Epoch 1/50
2700/2700 - 1s - loss: 0.6483 - accuracy: 0.6000 - val_loss: 0.5887 - val_accuracy: 0.6767 - 1s/epoch - 464us/sample


C:\Users\zrowx\anaconda3\lib\site-packages\keras\engine\training_v1.py:2333: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


Epoch 2/50
2700/2700 - 1s - loss: 0.5811 - accuracy: 0.6993 - val_loss: 0.5602 - val_accuracy: 0.6733 - 674ms/epoch - 250us/sample
Epoch 3/50
2700/2700 - 1s - loss: 0.5213 - accuracy: 0.7437 - val_loss: 0.5100 - val_accuracy: 0.7067 - 672ms/epoch - 249us/sample
Epoch 4/50
2700/2700 - 1s - loss: 0.4863 - accuracy: 0.7578 - val_loss: 0.5002 - val_accuracy: 0.7467 - 641ms/epoch - 237us/sample
Epoch 5/50
2700/2700 - 1s - loss: 0.4470 - accuracy: 0.7926 - val_loss: 0.4996 - val_accuracy: 0.7267 - 709ms/epoch - 263us/sample
Epoch 6/50
2700/2700 - 1s - loss: 0.3819 - accuracy: 0.8330 - val_loss: 0.5018 - val_accuracy: 0.7367 - 808ms/epoch - 299us/sample
Epoch 7/50
2700/2700 - 1s - loss: 0.3576 - accuracy: 0.8478 - val_loss: 0.5103 - val_accuracy: 0.7333 - 707ms/epoch - 262us/sample
Epoch 8/50
2700/2700 - 1s - loss: 0.3373 - accuracy: 0.8496 - val_loss: 0.5054 - val_accuracy: 0.7333 - 677ms/epoch - 251us/sample
Epoch 9/50
2700/2700 - 1s - loss: 0.3102 - accuracy: 0.8744 - val_loss: 0.5584 - va

In [32]:

# sample text to check if fake or not
X = """
The EU has stepped up its support to Turkey in the aftermath of the earthquake in the south-eastern province of Turkiye.
"""

In [33]:
# detection
sequences = tokenizer1.texts_to_sequences([X])[0]
sequences = pad_sequences([sequences], maxlen=54,
                          padding=padding_type, 
                          truncating=trunc_type)
if(model.predict(sequences, verbose=0)[0][0] >= 0.5):
    print("This news is True")
else:
    print("This news is false")

This news is false
